# Лабораторная работа 7. Управление ресурсами в однопроцессорной системе с неоднородными заявками

## Задание 1.

В однопроцессорную  систему случайным образом поступают на выполнение $m$ разных типов программ, отличающихся известной трудоемкостью $Q_1, Q_2, \ldots, Q_m$. Входящий поток простейший с интенсивностью $\lambda$.

Представив данную систему как одноканальную СМО с неограниченной очередью, вычислить среднее время обслуживания программ, считая длительность обслуживания случайной величиной (теоретически и экспериментально).

Реализовать алгоритм SPT, выбирая из очереди заявки в соответствии с их приоритетом по трудоемкости. Рассчитать среднее время обслуживания программ. Сравнить полученные результаты.

Реализовать алгоритм RR при заданном кванте времени $q$. Оценить среднее время обслуживания программ. Сравнить полученные результаты. Выяснить, как влияет величина кванта на среднее время обслуживания программ.

In [19]:
Variant<-1
set.seed(Variant) 
m<-sample(c(6:20),1)
lambda<-runif(1,0.1, 2)
# lambda = lambda - 1.4 # поменял по согласованию с преподавателем
Q<-rexp(m,0.3)
q<-sample(c(1:4),1)
data.frame(m,q, lambda)
print(Q)


m,q,lambda
14,4,0.8070354


 [1]  0.4856891  0.4659842  1.4535621  9.6498951  4.0985402  1.7989428
 [7]  3.1885583  0.4901533  4.6357838  2.5400995  4.1253452 14.7464474
[13]  3.5151439  3.4508132


### Одноканальная система с неограниченной очередью: теоретически

**среднее время обслуживания**
$$T_{\text{сист}}=\frac{1}{\mu(1-\rho)}\tag{лек-3}$$
здесь $\rho=\frac{\lambda}{\mu}$

In [17]:
mu <- 1/(sum(Q)/length(Q))
mu
T1_t <- 1/(mu*(1-(lambda/mu)))
T1_t

[1] 0.2075594

[1] 21.12096

### Одноканальная система с неограниченной очередью: экспериментально

In [12]:
Exp_time <- 100000 
cur_time <- 0
times_requests<-rexp(1,lambda)
times_execute<-0
queue <- c()
times_in_syst <- c()

In [13]:
while (cur_time < Exp_time) {
  if (length(queue) != 0 && cur_time > times_execute) {
    times_in_syst = append(times_in_syst, cur_time - queue[1])

    if (length(queue) < 2) {
      queue = c()
    } else {
      queue = queue[2:length(queue)]
    }

    if (length(queue) > 0) {
      times_execute = cur_time + rexp(1, mu)
    }
  }

  if (cur_time > times_requests) {
    if (length(queue) == 0) {
      times_execute = cur_time + rexp(1, mu)
    }

    queue = append(queue, cur_time)
    times_requests = cur_time + rexp(1, lambda)
  }

  cur_time = cur_time + 0.01
}

In [14]:
T1_e <- sum(times_in_syst) / length(times_in_syst)
T1_e

[1] 33969.94

### Алгоритм SPT

In [31]:
Exp_time <- 100000
cur_time <- 0
times_requests<-rexp(1,lambda)
times_execute=-1
times_in_syst <- c()
queue <- list()

In [32]:
while (cur_time < Exp_time) {
  if (times_execute != -1 && cur_time > times_execute) {
    times_execute = -1

    if (length(queue) > 0) {
      min_i <- 0
      min_qi <- 0
      min_t <- 0

      if (length(queue) == 1) {
        min_i = 1
        min_qi = queue[[1]][1]
        min_t = queue[[1]][2]
      } else {
        min_el <- queue[[1]]
        min_index <- 1
        for (i in 2:length(queue)) {
          if (queue[[i]][1] < min_el[1]) {
            min_el = queue[[i]]
            min_index = i
          }
        }

        min_i = min_index
        min_qi = min_el[1]
        min_t = min_el[2]
      }

      
      if (length(queue) < 2) {
        queue = list()
      } else {
        new_queue <- list()
        if (min_i > 1) {
          for (i in 1:(min_i-1)) {
            new_queue [[length(new_queue )+1]] = queue[[i]]
          }
        }

        if (length(queue) > min_i) {
          for (i in (min_i+1):length(queue)) {
            new_queue [[length(new_queue )+1]] = queue[[i]]
          }
        }
      }
      queue = new_queue

      
      times_execute = cur_time + min_qi
      times_in_syst = append(times_in_syst, (cur_time - min_t) + min_qi)
    }
  }

  if (cur_time > times_requests) {
    if (times_execute == -1) {
      times_execute = cur_time + sample(Q, 1, replace=T)

      times_in_syst = append(times_in_syst, times_execute - cur_time)
    } else {
      queue[[length(queue)+1]] = c(sample(Q, 1, replace=T), cur_time)
    }
    
    times_requests = cur_time + rexp(1, lambda)
  }

  cur_time = cur_time + 0.01
}

In [33]:
T2 <- sum(times_in_syst) / length(times_in_syst)
T2

[1] 32.53188

### Алгоритм RR

In [34]:
Exp_time <- 100000
cur_time <- 0
times_requests<-rexp(1,lambda)
times_execute<-c()
times_in_syst <- c()
queue <- list()

q_cur <- 0

# сравнение квантов
# q <- 1 # 2, 3, 4

In [35]:
DeleteElemOfList<-function(lst, index) {
  new_lst <- list()
  if (length(lst) < 2) return (new_lst)

  if (index > 1) {
    for (i in 1:(index-1)) {
      new_lst[[length(new_lst)+1]] = lst[[i]]
    }
  }

  if (length(lst) > index) {
    for (i in (index+1):length(lst)) {
      new_lst[[length(new_lst)+1]] = lst[[i]]
    }
  }

  return (new_lst)
}

while (cur_time < Exp_time) {
  if (cur_time > q_cur && length(queue) && length(times_execute)) {
    deleted_element <- times_execute
    queue[[length(queue)+1]] = c(deleted_element[1] - q, deleted_element[2])
    
    qq <- queue[[1]]
    queue = DeleteElemOfList(queue, 1)
    times_execute = c(qq[1], qq[2], qq[1] + cur_time)
    q_cur<- (cur_time + q)
  }


  if (cur_time> times_execute[3] && length(times_execute)) {
    times_in_syst = append(times_in_syst, cur_time- times_execute[2])
    times_execute = c()

    if (length(queue)) {
      qq <- queue[[1]]
      queue = DeleteElemOfList(queue, 1)
      times_execute = c(qq[1], qq[2], qq[1] + cur_time)
      q_cur<- (cur_time+ q)
      
    }
  }

  if (cur_time > times_requests) {
    if (length(times_execute)) {
      queue[[length(queue)+1]] = c(sample(Q, 1, replace=T), cur_time)
    } else {
      element <- sample(Q, 1, replace=T)
      times_execute = c(element, cur_time, element+cur_time)
      q_cur<- (cur_time + q)
    }
    
    times_requests = cur_time+ rexp(1, lambda)
  }

  cur_time = cur_time + 0.01
}

In [36]:
T3 <- sum(times_in_syst) / length(times_in_syst)
T3

[1] 23.29068

При тестировании квантов, то при малых значениях q, результат выходит не очень хорошим

### Вывод

одноканальная с неограниченной очередью (эксперимент, теория)

In [37]:
T1_t; T1_e

[1] 21.12096

[1] 20.96186

SPT

In [38]:
T2

[1] 32.53188

RR

In [39]:
T3

[1] 23.29068

Лучше всего справился метод RR